<a href="https://colab.research.google.com/github/zwcrowley/module_21_deep_learning_challenge/blob/main/deep_learning_charity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Project 4: Predicting Congressional Bill Passage**

# Senate Model: Machine Learning Optimization and Model Output

## Team 7


## Import dependencies and read in data:

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from joblib import dump, load
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#  Import and read the cleaned house data from AWS S3:
senate_df = pd.read_csv("https://raw.githubusercontent.com/JJERANEK/Project-4/main/Resources/senate_cleaned.csv")
senate_df.head()

,Bill Type,Congress,Number of Cosponsors,Cosponsor Dems,Cosponsor Reps,Cosponsor Ind,Cosponsor States,Committees,Latest Action,Subject,...,International Narcotics Control,Ethics,Indian Affairs,Intelligence,Printing,Taxation,Library,Economic,bill_passed,bill_referred_committee
0,S.,113,15,15,0,0,12,Senate - Judiciary,Read twice and referred to the Committee on th...,Border security and unlawful immigration,...,0,0,0,0,0,0,0,0,0,1
1,S.,113,16,16,0,0,12,Senate - Judiciary,Read twice and referred to the Committee on th...,Child health,...,0,0,0,0,0,0,0,0,0,1
2,S.,113,16,15,0,1,11,"Senate - Health, Education, Labor, and Pensions",Read twice and referred to the Committee on He...,Child safety and welfare,...,0,0,0,0,0,0,0,0,0,1
3,S.,113,14,14,0,0,12,"Senate - Commerce, Science, and Transportation",Read twice and referred to the Committee on Co...,Academic performance and assessments,...,0,0,0,0,0,0,0,0,0,1
4,S.,113,31,30,0,1,24,Senate - Judiciary,Read twice and referred to the Committee on th...,Crime and Law Enforcement,...,0,0,0,0,0,0,0,0,0,1


In [3]:
# Import from google drive folder:
# Mount google drive to get data:
from google.colab import drive
drive.mount('/content/gdrive')
# senate_filepath = "/content/gdrive/MyDrive/DataClassNotebooks/Project-4/Resources/senate_cleaned.csv"

# Read in senate data using pandas:
# senate_df = pd.read_csv(senate_filepath)
# Glimpse senate data:
# senate_df.head()

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# Senate Model

## Preprocessing:

In [4]:
# Check for NAs, duplicates and get the shape of the data:
print(f'The shape of the senate_df data is: {senate_df.shape}')
print(f'The number of NAs in the senate_df data: {senate_df.isnull().sum()}')
print(f'The duplicate rows of NAs in the senate_df data: {senate_df.duplicated().sum()}')
# There are 40 columns, 15593 rows, and no NAs or duplicates in the senate_df dataset.
# target is bill_passed

The shape of the senate_df data is: (14780, 40)
The number of NAs in the senate_df data: Bill Type                                       0
Congress                                        0
Number of Cosponsors                            0
Cosponsor Dems                                  0
Cosponsor Reps                                  0
Cosponsor Ind                                   0
Cosponsor States                                0
Committees                                    296
Latest Action                                   0
Subject                                         1
Sponsor Title                                   0
Sponsor Party                                   0
Sponsor State                                   0
Month Introduced                                0
Agriculture, Nutrition, and Forestry            0
Appropriations                                  0
Armed Services                                  0
Banking, Housing, and Urban Affairs             0
Budget     

In [5]:
# Numeric variable stats
senate_df.describe()

,Congress,Number of Cosponsors,Cosponsor Dems,Cosponsor Reps,Cosponsor Ind,Cosponsor States,Month Introduced,"Agriculture, Nutrition, and Forestry",Appropriations,Armed Services,...,International Narcotics Control,Ethics,Indian Affairs,Intelligence,Printing,Taxation,Library,Economic,bill_passed,bill_referred_committee
count,14780.000000,14780.000000,14780.000000,14780.000000,14780.000000,14780.000000,14780.000000,14780.000000,14780.000000,14780.000000,...,14780.000000,14780.000000,14780.000000,14780.000000,14780.000000,14780.000000,14780.000000,14780.000000,14780.000000,14780.000000
mean,114.781597,5.389107,2.818403,2.464750,0.105954,4.411908,5.608390,0.045737,0.028078,0.048850,...,0.020027,0.020027,0.035521,0.022530,0.020027,0.020027,0.020027,0.020027,0.030785,0.777673
std,1.321436,9.364853,6.002123,5.545356,0.345882,6.607823,3.206763,0.208922,0.165203,0.215561,...,0.140097,0.140097,0.185099,0.148406,0.140097,0.140097,0.140097,0.140097,0.172740,0.415824
min,113.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,114.000000,1.000000,0.000000,0.000000,0.000000,1.000000,3.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,115.000000,2.000000,1.000000,1.000000,0.000000,2.000000,5.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,116.000000,6.000000,3.000000,2.000000,0.000000,5.000000,8.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,117.000000,99.000000,55.000000,53.000000,2.000000,50.000000,12.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [6]:
# Get the column names:
list(senate_df.columns)


['Bill Type',
 'Congress',
 'Number of Cosponsors',
 'Cosponsor Dems',
 'Cosponsor Reps',
 'Cosponsor Ind',
 'Cosponsor States',
 'Committees',
 'Latest Action',
 'Subject',
 'Sponsor Title',
 'Sponsor Party',
 'Sponsor State',
 'Month Introduced',
 'Agriculture, Nutrition, and Forestry',
 'Appropriations',
 'Armed Services',
 'Banking, Housing, and Urban Affairs',
 'Budget',
 'Commerce, Science, and Transportation',
 'Energy and Natural Resources',
 'Environment and Public Works',
 'Finance',
 'Foreign Relations',
 'Health, Education, Labor, and Pensions',
 'Homeland Security and Governmental Affairs',
 'Judiciary',
 'Rules and Administration',
 'Small Business and Entrepreneurship',
 'Veterans Affairs',
 'International Narcotics Control',
 'Ethics',
 'Indian Affairs',
 'Intelligence',
 'Printing',
 'Taxation',
 'Library',
 'Economic',
 'bill_passed',
 'bill_referred_committee']

In [7]:
# Check out Subject column:
print(f'Count of values for Subject column: \n{senate_df["Subject"].value_counts()}')

Count of values for Subject column: 
Administrative law and regulatory procedures    828
Armed Forces and National Security              827
Health                                          722
Taxation                                        503
Crime and Law Enforcement                       376
                                               ... 
National and community service                    1
Earth sciences                                    1
Bolivia                                           1
Commodities markets                               1
Nuclear Regulatory Commission (NRC)               1
Name: Subject, Length: 688, dtype: int64


In [8]:
# Model Target:
senate_df["bill_passed"].value_counts()
# 455 bills  in the dataset which originated from the House became law in the 113th-117th Congresses

0    14325
1      455
Name: bill_passed, dtype: int64

In [9]:
# Drop the non-beneficial columns: 'Unnamed' column.
senate_df = senate_df.drop(["Bill Type", "Committees", "Latest Action" ], axis='columns')
senate_df.head()

,Congress,Number of Cosponsors,Cosponsor Dems,Cosponsor Reps,Cosponsor Ind,Cosponsor States,Subject,Sponsor Title,Sponsor Party,Sponsor State,...,International Narcotics Control,Ethics,Indian Affairs,Intelligence,Printing,Taxation,Library,Economic,bill_passed,bill_referred_committee
0,113,15,15,0,0,12,Border security and unlawful immigration,Sen.,D,NV,...,0,0,0,0,0,0,0,0,0,1
1,113,16,16,0,0,12,Child health,Sen.,D,NV,...,0,0,0,0,0,0,0,0,0,1
2,113,16,15,0,1,11,Child safety and welfare,Sen.,D,NV,...,0,0,0,0,0,0,0,0,0,1
3,113,14,14,0,0,12,Academic performance and assessments,Sen.,D,NV,...,0,0,0,0,0,0,0,0,0,1
4,113,31,30,0,1,24,Crime and Law Enforcement,Sen.,D,NV,...,0,0,0,0,0,0,0,0,0,1


In [10]:
senate_df.dtypes
# Need to create dummies for: Subject, Sponsor Title,  Sponsor Party, Sponsor State columns
# Need to bin Subject column before creating dummy vars.

Congress                                       int64
Number of Cosponsors                           int64
Cosponsor Dems                                 int64
Cosponsor Reps                                 int64
Cosponsor Ind                                  int64
Cosponsor States                               int64
Subject                                       object
Sponsor Title                                 object
Sponsor Party                                 object
Sponsor State                                 object
Month Introduced                               int64
Agriculture, Nutrition, and Forestry           int64
Appropriations                                 int64
Armed Services                                 int64
Banking, Housing, and Urban Affairs            int64
Budget                                         int64
Commerce, Science, and Transportation          int64
Energy and Natural Resources                   int64
Environment and Public Works                  

In [11]:
# Determine the number of unique values in each column.
senate_df.nunique()

Congress                                        5
Number of Cosponsors                           86
Cosponsor Dems                                 53
Cosponsor Reps                                 53
Cosponsor Ind                                   3
Cosponsor States                               51
Subject                                       688
Sponsor Title                                   1
Sponsor Party                                   3
Sponsor State                                  50
Month Introduced                               12
Agriculture, Nutrition, and Forestry            2
Appropriations                                  2
Armed Services                                  2
Banking, Housing, and Urban Affairs             2
Budget                                          2
Commerce, Science, and Transportation           2
Energy and Natural Resources                    2
Environment and Public Works                    2
Finance                                         2


In [12]:
# Look at Subject value counts for binning
Subject_counts_sen = senate_df["Subject"].value_counts()
print(f'Count of values for Subject column: \n{Subject_counts_sen}')

Count of values for Subject column: 
Administrative law and regulatory procedures    828
Armed Forces and National Security              827
Health                                          722
Taxation                                        503
Crime and Law Enforcement                       376
                                               ... 
National and community service                    1
Earth sciences                                    1
Bolivia                                           1
Commodities markets                               1
Nuclear Regulatory Commission (NRC)               1
Name: Subject, Length: 688, dtype: int64


In [13]:
# Now change cutoff value to get  bins for Subject
# Cutoff value of 100 to bin: 
# use the variable name `Subject_types_to_replace_sen`
Subject_types_to_replace_sen = list(Subject_counts_sen[Subject_counts_sen<100].index)

# Replace in dataframe
for sub in Subject_types_to_replace_sen:
    senate_df['Subject'] = senate_df['Subject'].replace(sub,"Other")

# Check to make sure binning was successful
senate_df['Subject'].value_counts()
# 27 Bins

Other                                           7410
Administrative law and regulatory procedures     828
Armed Forces and National Security               827
Health                                           722
Taxation                                         503
Crime and Law Enforcement                        376
Government Operations and Politics               364
Congressional oversight                          313
Public Lands and Natural Resources               310
International Affairs                            280
Finance and Financial Sector                     278
Education                                        260
Transportation and Public Works                  254
Commerce                                         239
Science, Technology, Communications              198
Agriculture and Food                             176
Advisory bodies                                  176
Energy                                           170
Appropriations                                

In [14]:
# Convert categorical data to numeric with `pd.get_dummies`
senate_df = pd.get_dummies(senate_df,dtype=float)
senate_df.head()
# Creates a df of 114 columns

,Congress,Number of Cosponsors,Cosponsor Dems,Cosponsor Reps,Cosponsor Ind,Cosponsor States,Month Introduced,"Agriculture, Nutrition, and Forestry",Appropriations,Armed Services,...,Sponsor State_SD,Sponsor State_TN,Sponsor State_TX,Sponsor State_UT,Sponsor State_VA,Sponsor State_VT,Sponsor State_WA,Sponsor State_WI,Sponsor State_WV,Sponsor State_WY
0,113,15,15,0,0,12,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,113,16,16,0,0,12,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,113,16,15,0,1,11,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,113,14,14,0,0,12,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,113,31,30,0,1,24,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
# Split our preprocessed data into our features and target arrays
y = senate_df["bill_passed"].values
X = senate_df.drop(["bill_passed"], axis='columns').values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [16]:
# Create a StandardScaler instances
senate_scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = senate_scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

### Attempt #1


*   Total layers: 5 total
*   Activation function to for each layer is: relu, relu, tanh, tanh,  sigmoid
*   Number of neurons for each hidden layers: 9,7,7,5,1
*   Epochs: 100

In [17]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
# Define the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  9
hidden_nodes_layer2_3 = 7
hidden_nodes_layer4 = 5

nn_sen = tf.keras.models.Sequential()

# First hidden layer
nn_sen.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn_sen.add(tf.keras.layers.Dense(units=hidden_nodes_layer2_3, activation="relu"))

# Third hidden layer
nn_sen.add(tf.keras.layers.Dense(units=hidden_nodes_layer2_3, activation="tanh"))

# Fourth hidden layer
nn_sen.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="tanh"))

# Output layer
nn_sen.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_sen.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 9)                 1017      
                                                                 
 dense_1 (Dense)             (None, 7)                 70        
                                                                 
 dense_2 (Dense)             (None, 7)                 56        
                                                                 
 dense_3 (Dense)             (None, 5)                 40        
                                                                 
 dense_4 (Dense)             (None, 1)                 6         
                                                                 
Total params: 1,189
Trainable params: 1,189
Non-trainable params: 0
_________________________________________________________________


In [18]:
# Compile the model
nn_sen.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [19]:
# Train the model
fit_model = nn_sen.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
347/347 [==============================] - 5s 7ms/step - loss: 0.2255 - accuracy: 0.9489
Epoch 2/100
347/347 [==============================] - 3s 9ms/step - loss: 0.1322 - accuracy: 0.9678
Epoch 3/100
347/347 [==============================] - 3s 7ms/step - loss: 0.1074 - accuracy: 0.9678
Epoch 4/100
347/347 [==============================] - 1s 4ms/step - loss: 0.0952 - accuracy: 0.9677
Epoch 5/100
347/347 [==============================] - 2s 5ms/step - loss: 0.0885 - accuracy: 0.9676
Epoch 6/100
347/347 [==============================] - 2s 5ms/step - loss: 0.0841 - accuracy: 0.9681
Epoch 7/100
347/347 [==============================] - 1s 4ms/step - loss: 0.0803 - accuracy: 0.9687
Epoch 8/100
347/347 [==============================] - 1s 2ms/step - loss: 0.0782 - accuracy: 0.9687
Epoch 9/100
347/347 [==============================] - 1s 2ms/step - loss: 0.0750 - accuracy: 0.9691
Epoch 10/100
347/347 [==============================] - 1s 2ms/step - loss: 0.0722 - accura

In [20]:
# Evaluate the Senate model using the test data:
model_loss_1, model_accuracy_1 = nn_sen.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {round(model_loss_1,4)}, Accuracy: {round(model_accuracy_1,4)}")

116/116 - 0s - loss: 0.1631 - accuracy: 0.9645 - 400ms/epoch - 3ms/step
Loss: 0.1631, Accuracy: 0.9645


In [21]:
# Export the model from optimization attempt 2 to HDF5 file
from google.colab import files

output_filepath = "/content/gdrive/MyDrive/DataClassNotebooks/Project-4/output"

# Save model 2, nn_2, and download a copy to local machine:
nn_sen.save(f'{output_filepath}/senate_model.h5')
files.download(f'{output_filepath}/senate_model.h5')

# Save the StandardScaler() instance, senate_scaler, for use in the flask app later:
dump(senate_scaler, f'{output_filepath}/senate_scaler.bin', compress=True)
files.download(f'{output_filepath}/senate_scaler.bin')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>